In [1]:
from simulator import combat_sim
from simulator import test_units
from simulator import get_health
from simulator import get_damage

In [2]:
Units = test_units()

In [3]:
# test enemy army
enemy_army_comp = {}
enemy_army_comp['SCV'] = 10
enemy_army_comp['Marine'] = 10
enemy_army_comp['Medivac'] = 5

# test test_army
test_army_comp = {}
test_army_comp['SCV'] = 10
test_army_comp['Marine'] = 12
test_army_comp['Medivac'] = 5

In [4]:
combat_sim(enemy_army_comp, test_army_comp)

[Marine_HP:25.400000000000002,
 Marine_HP:35.2,
 Marine_HP:35.2,
 Marine_HP:13.73,
 Marine_HP:20.73,
 Marine_HP:1.1299999999999955,
 Marine_HP:15.600000000000001,
 Marine_HP:16.059999999999995,
 Marine_HP:9.729999999999993,
 Marine_HP:5.850000000000019,
 Marine_HP:35.2]

Function in our linear program to define "viability". By default, viability is described as an army comp that has at least 10% of its units remaining at least 90% of the time. Instead of just unit count, we might want to measure viability using total army health or damage. For example, if we have an army composed of 10 Marines and 1 Battlecruiser and only the Battlecruiser survives, we might still want to count that army as viable since that single Battlecruiser can do much more damage than a small group of Marines.

In [5]:
def viable(enemy_comp, test_comp, test_type='units'):
    """
    Input is the enemy army composition, the army composition that
    we are testing, and the type of viablity test we are running.
    test_type is a string, either "units", "health", or "damage"
    Runs simulation 200 times. If the test army has at
    least 10% of its test type remaining 90% of the time, return 1
    Otherwise return 0
    """
    test_limit = 0
    if test_type == 'units':
        unit_count = 0
        for unit in test_comp:
            unit_count += test_comp[unit]
        test_limit = 0.1 * unit_count
    
    if test_type == 'health':
        army_health = 0
        for unit in test_comp:
            unit_hp = Units[unit]['hp']
            army_health += unit_hp * test_comp[unit]
        test_limit = 0.1 * army_health
    if test_type == 'damage':
        army_dmg = 0
        for unit in test_comp:
            unit_dmg = Units[unit]['dps']
            army_dmg += unit_dmg * test_comp[unit]
        test_limit = 0.1 * army_dmg
    
    wins = 0
    sim_count = 200
    sims = sim_count
    while sims > 0:
        test_army = combat_sim(enemy_comp, test_comp)
        if (test_type == 'units') and (len(test_army) > test_limit):
            wins += 1
        elif (test_type == 'health') and (get_health(test_army) > test_limit):
            wins += 1
        elif (test_type == 'damage') and (get_damage(test_army) > test_limit):
            wins += 1
        sims -= 1
    if (wins / sim_count) > 0.9:
        return 1
    else:
        return 0

In [7]:
viable(enemy_army_comp, test_army_comp, 'damage')

1